In [ ]:
#!/usr/bin/env python3
"""
Bincom ICT Python Developer Interview Solution
Analyzing dress colors worn by staff throughout the week
"""

import re
import random
import statistics
from collections import Counter
import psycopg2
from psycopg2 import sql
import math

# Raw data from HTML table
data = {
    'MONDAY': 'GREEN, YELLOW, GREEN, BROWN, BLUE, PINK, BLUE, YELLOW, ORANGE, CREAM, ORANGE, RED, WHITE, BLUE, WHITE, BLUE, BLUE, BLUE, GREEN',
    'TUESDAY': 'ARSH, BROWN, GREEN, BROWN, BLUE, BLUE, BLEW, PINK, PINK, ORANGE, ORANGE, RED, WHITE, BLUE, WHITE, WHITE, BLUE, BLUE, BLUE',
    'WEDNESDAY': 'GREEN, YELLOW, GREEN, BROWN, BLUE, PINK, RED, YELLOW, ORANGE, RED, ORANGE, RED, BLUE, BLUE, WHITE, BLUE, BLUE, WHITE, WHITE',
    'THURSDAY': 'BLUE, BLUE, GREEN, WHITE, BLUE, BROWN, PINK, YELLOW, ORANGE, CREAM, ORANGE, RED, WHITE, BLUE, WHITE, BLUE, BLUE, BLUE, GREEN',
    'FRIDAY': 'GREEN, WHITE, GREEN, BROWN, BLUE, BLUE, BLACK, WHITE, ORANGE, RED, RED, RED, WHITE, BLUE, WHITE, BLUE, BLUE, BLUE, WHITE'
}

def extract_colors():
    """Extract and clean all colors from the data"""
    all_colors = []
    
    for day, colors_str in data.items():
        # Split by comma and clean each color
        colors = [color.strip().upper() for color in colors_str.split(',')]
        
        # Clean up typos and inconsistencies
        for color in colors:
            if color == 'ARSH':  # Likely meant ASH
                color = 'ASH'
            elif color == 'BLEW':  # Likely meant BLUE
                color = 'BLUE'
            all_colors.append(color)
    
    return all_colors

def get_color_frequencies():
    """Get frequency count of all colors"""
    colors = extract_colors()
    return Counter(colors)

# 1. Which color of shirt is the mean color?
def get_mean_color():
    """
    For categorical data like colors, mean is typically the mode (most frequent)
    """
    frequencies = get_color_frequencies()
    mean_color = frequencies.most_common(1)[0][0]
    return mean_color

# 2. Which color is mostly worn throughout the week?
def get_most_worn_color():
    """Get the most frequently worn color"""
    frequencies = get_color_frequencies()
    return frequencies.most_common(1)[0]

# 3. Which color is the median?
def get_median_color():
    """
    For categorical data, median is the middle value when sorted by frequency
    """
    colors = extract_colors()
    frequencies = get_color_frequencies()
    
    # Sort colors by frequency
    sorted_colors = sorted(frequencies.items(), key=lambda x: x[1])
    
    # Get median position
    n = len(sorted_colors)
    median_index = n // 2
    
    if n % 2 == 0:
        # Even number of unique colors - take average of two middle values
        # For categorical data, we'll take the lower middle value
        median_color = sorted_colors[median_index - 1][0]
    else:
        # Odd number - take middle value
        median_color = sorted_colors[median_index][0]
    
    return median_color

# 4. BONUS: Get the variance of the colors
def get_color_variance():
    """Calculate variance of color frequencies"""
    frequencies = get_color_frequencies()
    frequency_values = list(frequencies.values())
    
    if len(frequency_values) > 1:
        variance = statistics.variance(frequency_values)
    else:
        variance = 0
    
    return variance

# 5. BONUS: Probability that a randomly chosen color is red
def get_red_probability():
    """Calculate probability of selecting red color randomly"""
    colors = extract_colors()
    red_count = colors.count('RED')
    total_count = len(colors)
    
    probability = red_count / total_count if total_count > 0 else 0
    return probability

# 6. Save colors and frequencies to PostgreSQL database
def save_to_database():
    """Save color frequencies to PostgreSQL database"""
    frequencies = get_color_frequencies()
    
    try:
        # Database connection parameters (adjust as needed)
        conn_params = {
            'host': 'localhost',
            'database': 'bincom_colors',
            'user': 'your_username',
            'password': 'your_password'
        }
        
        # Connect to database
        conn = psycopg2.connect(**conn_params)
        cur = conn.cursor()
        
        # Create table if it doesn't exist
        create_table_query = """
        CREATE TABLE IF NOT EXISTS color_frequencies (
            id SERIAL PRIMARY KEY,
            color VARCHAR(50) NOT NULL,
            frequency INTEGER NOT NULL,
            created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP
        );
        """
        cur.execute(create_table_query)
        
        # Clear existing data
        cur.execute("DELETE FROM color_frequencies;")
        
        # Insert color frequencies
        for color, frequency in frequencies.items():
            insert_query = """
            INSERT INTO color_frequencies (color, frequency) 
            VALUES (%s, %s);
            """
            cur.execute(insert_query, (color, frequency))
        
        # Commit changes
        conn.commit()
        print("Color frequencies saved to database successfully!")
        
        # Close connections
        cur.close()
        conn.close()
        
    except psycopg2.Error as e:
        print(f"Database error: {e}")
        print("Note: Make sure PostgreSQL is installed and running, and database credentials are correct.")

# 7. BONUS: Recursive searching algorithm
def recursive_search(arr, target, index=0):
    """
    Recursive binary search algorithm
    Assumes the array is sorted
    """
    if index >= len(arr):
        return -1
    
    if arr[index] == target:
        return index
    
    return recursive_search(arr, target, index + 1)

def recursive_binary_search(arr, target, left=0, right=None):
    """
    More efficient recursive binary search
    """
    if right is None:
        right = len(arr) - 1
    
    if left > right:
        return -1
    
    mid = (left + right) // 2
    
    if arr[mid] == target:
        return mid
    elif arr[mid] > target:
        return recursive_binary_search(arr, target, left, mid - 1)
    else:
        return recursive_binary_search(arr, target, mid + 1, right)

# 8. Generate random 4-digit binary number and convert to base 10
def generate_binary_to_decimal():
    """Generate random 4-digit binary number and convert to decimal"""
    # Generate random 4-digit binary number
    binary_digits = [str(random.randint(0, 1)) for _ in range(4)]
    binary_string = ''.join(binary_digits)
    
    # Convert to decimal
    decimal_value = int(binary_string, 2)
    
    return binary_string, decimal_value

# 9. Sum of first 50 Fibonacci numbers
def fibonacci_sum(n=50):
    """Calculate sum of first n Fibonacci numbers"""
    if n <= 0:
        return 0
    elif n == 1:
        return 0
    elif n == 2:
        return 1
    
    # Generate first n Fibonacci numbers
    fib_sequence = [0, 1]
    
    for i in range(2, n):
        next_fib = fib_sequence[i-1] + fib_sequence[i-2]
        fib_sequence.append(next_fib)
    
    return sum(fib_sequence), fib_sequence

def main():
    """Main function to run all analyses"""
    print("=" * 60)
    print("BINCOM ICT PYTHON DEVELOPER INTERVIEW SOLUTION")
    print("=" * 60)
    
    # Extract and display basic info
    colors = extract_colors()
    frequencies = get_color_frequencies()
    
    print(f"\nTotal colors recorded: {len(colors)}")
    print(f"Unique colors: {len(frequencies)}")
    print(f"\nColor frequencies:")
    for color, freq in frequencies.most_common():
        print(f"  {color}: {freq}")
    
    print("\n" + "=" * 60)
    print("ANSWERS TO INTERVIEW QUESTIONS")
    print("=" * 60)
    
    # 1. Mean color
    mean_color = get_mean_color()
    print(f"\n1. Mean color (most frequent): {mean_color}")
    
    # 2. Most worn color
    most_worn = get_most_worn_color()
    print(f"\n2. Most worn color: {most_worn[0]} (worn {most_worn[1]} times)")
    
    # 3. Median color
    median_color = get_median_color()
    print(f"\n3. Median color: {median_color}")
    
    # 4. Variance
    variance = get_color_variance()
    print(f"\n4. BONUS - Variance of color frequencies: {variance:.2f}")
    
    # 5. Red probability
    red_prob = get_red_probability()
    print(f"\n5. BONUS - Probability of red color: {red_prob:.4f} ({red_prob*100:.2f}%)")
    
    # 6. Database (demonstrate the function)
    print(f"\n6. Database function created (save_to_database())")
    print("   Note: Requires PostgreSQL setup and proper credentials")
    
    # 7. Recursive search demonstration
    print(f"\n7. BONUS - Recursive search algorithm:")
    test_numbers = [1, 3, 5, 7, 9, 11, 13, 15, 17, 19]
    target = int(input("   Enter a number to search for: ") or "7")
    result = recursive_binary_search(test_numbers, target)
    if result != -1:
        print(f"   Number {target} found at index {result}")
    else:
        print(f"   Number {target} not found in the list")
    
    # 8. Binary to decimal conversion
    print(f"\n8. Random 4-digit binary to decimal conversion:")
    for i in range(3):
        binary, decimal = generate_binary_to_decimal()
        print(f"   Binary: {binary} -> Decimal: {decimal}")
    
    # 9. Fibonacci sum
    print(f"\n9. Sum of first 50 Fibonacci numbers:")
    fib_sum, fib_sequence = fibonacci_sum(50)
    print(f"   Sum: {fib_sum}")
    print(f"   First 10 numbers: {fib_sequence[:10]}")
    print(f"   Last 10 numbers: {fib_sequence[-10:]}")

if __name__ == "__main__":
    main()